In [24]:
import re
import numpy as np
import pandas as pd

In [26]:
STOP_WORDS = {
"i",
"me",
"my",
"myself",
"we",
"our",
"ours",
"ourselves",
"you",
"you're",
"you've",
"you'll",
"you'd",
"your",
"yours",
"yourself",
"yourselves",
"he",
"him",
"his",
"himself",
"she",
"she's",
"her",
"hers",
"herself",
"it",
"it's",
"its",
"itself",
"they",
"them",
"their",
"theirs",
"themselves",
"what",
"which",
"who",
"whom",
"this",
"that",
"that'll",
"these",
"those",
"am",
"is",
"are",
"was",
"were",
"be"}

In [7]:
df = pd.read_csv('td_matrix.csv')
df.set_index(keys='Unnamed: 0', inplace=True)
df

,https://psgtech.edu,https://psgtech.edu/1974Scholarship.php,https://psgtech.edu/Academ.php,https://psgtech.edu/Accsec.php,https://psgtech.edu/AluScholarships.php,https://psgtech.edu/Antiragging.php,https://psgtech.edu/CMC.php,https://psgtech.edu/COE/FAQ-Exam%20Fees.html,https://psgtech.edu/COE/FAQ-Exam.html,https://psgtech.edu/COE/FAQ-Gradesheet.html,...,https://www.psgtech.edu/training/overseas.htm,https://www.psgtech.edu/training/padmanaban.htm,https://www.psgtech.edu/training/phinternship.html,https://www.psgtech.edu/training/ravishankar.htm,https://www.psgtech.edu/training/sathees.htm,https://www.psgtech.edu/training/sathya.htm,https://www.psgtech.edu/training/upd.html,https://www.psgtech.edu/training/velusamy.htm,https://www.psgtech.edu/training/vsc.htm,https://www.psgtech.edu/trustees.php
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01092022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [223]:
def cosineSimilarity(queryWeights, tdMatrix):
    return np.dot(tdMatrix, queryWeights) / (np.linalg.norm(queryWeights) * np.linalg.norm(tdMatrix, axis=1))

def findTopNRelevantDocsWithCosineSimilarity(query_weights, tdMatrixDF, N):
    cosine_similarity_scores = cosineSimilarity(query_weights, tdMatrixDF.T.values)

    # Sort in descending order of cosine similarity
    df = pd.DataFrame({'docID': tdMatrixDF.columns, 'cosineSimilarity': cosine_similarity_scores.flatten()})
    sorted_df = df.sort_values(by='cosineSimilarity', ascending=False)

    # Return the top N relevant documents from the sorted dataframe
    return sorted_df['docID'].values[:N].tolist()

def prompt_query():
    query = input()

    vectorizer = TfidfVectorizer()
    stemmer = PorterStemmer()

    punctuation_removed_query = re.sub(r'[^\w\s]', '', query)
    clean_query = re.sub(r'\s+',' ',punctuation_removed_query.strip())
    processed_query = [stemmer.stem(token.lower()) for token in word_tokenize(clean_query)]

    query_vector = vectorizer.fit_transform([' '.join(processed_query)])
    qf = pd.DataFrame(query_vector.toarray(), columns=vectorizer.get_feature_names_out())

    full_vector = [qf[col][0] if col in vectorizer.get_feature_names_out() else 0 for col in df.index]
    return findTopNRelevantDocsWithCosineSimilarity(full_vector, df, 10)

In [224]:
prompt_query()

['https://psgtech.edu/programme.php',
 'https://www.psgtech.edu/programme.php',
 'https://psgtech.edu/programme_offered.php',
 'https://www.psgtech.edu/programme_offered.php',
 'https://psgtech.edu/Continuing-on-going.php',
 'https://www.psgtech.edu/Continuing-on-going.php',
 'https://psgtech.edu/cnld/',
 'https://www.psgtech.edu/cnld/',
 'https://psgtech.edu/cnld/index.html',
 'https://www.psgtech.edu/cnld/index.html']